前一个版本实现了地铁站的POI抓取，这个脚本专门做时间提取及其 Debug

In [1]:
import urllib.request #发送请求
from urllib import parse #URL编码
import json #解析json数据
import jsonpath #提取json数据
import pandas as pd #导入pandas库
import numpy as np
import pickle

In [2]:
stationInfo = pd.read_csv('../../Metro/station_transInfo_cleaned.csv')
stationInfo

,Unnamed: 0,stationID,lon,lat,name,includeStation,neighborStation
0,0,2048,121.439716,31.171622,龙漕路,"[313, 1228]","[312, 314, 2012, 2038]"
1,1,2049,121.527849,31.224457,浦电路,"[418, 631]","[2010, 2015]"
2,2,2050,121.441840,31.225058,静安寺,"[243, 739]","[2041, 738, 2017, 2054]"
3,3,2051,121.422616,31.188487,宜山路,"[315, 402, 933]","[932, 2025, 2029, 314, 2044]"
4,4,2052,121.552971,31.205156,龙阳路,"[252, 752, 1621]","[751, 753, 1622, 251, 253]"
...,...,...,...,...,...,...,...
297,317,2043,121.412622,31.220079,中山公园,"[241, 318, 405]","[240, 2046, 2009, 2054]"
298,318,2044,121.416178,31.199283,虹桥路,"[316, 403, 1049]","[1048, 2051, 2037, 2046]"
299,319,509,121.415276,31.020130,东川路,NaN,"[508, 510]"
300,320,2046,121.412433,31.211456,延安西路,"[317, 404]","[2043, 2044]"


In [3]:
dict_staionName = dict(zip(stationInfo['stationID'], stationInfo['name']))
dict_stationLoc = dict(zip(stationInfo['stationID'], zip(stationInfo['lon'],stationInfo['lat'])))
stationID_List = list(stationInfo['stationID'])

travelTime = np.zeros((len(stationID_List), len(stationID_List)))

In [4]:
LBSkey = '5f4a9713e41c83183b72817fb3acb428'
POIkey = '9ad58ff9754b6fe3c7e51f43c06f453f'
mykey = 'd63d7e23659a45277696c954da708d42'

In [5]:
with open('TempData/dict_stationPOI.pkl', 'rb') as f:
    dict_stationPOI = pickle.load(f)
len(dict_stationPOI)

302

In [6]:
dict_Name2ID = dict(zip(stationInfo['name'], stationInfo['stationID']))
id1 = dict_Name2ID['外环路']
id2 = dict_Name2ID['迪士尼']
id3 = dict_Name2ID['浦东国际机场']
id1, id2, id3


(112, 1163, 263)

这三个站点提取出的时间为空，怀疑是POI搞错了


手动搜索并更正POI

In [ ]:
stationID = id1 # 外环路
url='https://restapi.amap.com/v3/place/text?'
params = {
    'key': mykey,
    'keywords': "外环路（地铁站）",
    'types':150000,
    'location': str(dict_stationLoc[stationID][0])[:9] + ',' + str(dict_stationLoc[stationID][1])[:8],
    'city':'021',
}
params = parse.urlencode(params)
url = url + params
#print(url)
response = urllib.request.urlopen(url)
content = response.read().decode('utf-8')
data = json.loads(content)
data

In [ ]:
dict_stationPOI[id1]

应为 BV10039895 而非 BV10035539，猜想得证

In [ ]:
stationID = id2 # 迪士尼（地铁站）
url='https://restapi.amap.com/v3/place/text?'
params = {
    'key': mykey,
    'keywords': "迪士尼（地铁站）",
    'types':150000,
    'location': str(dict_stationLoc[stationID][0])[:9] + ',' + str(dict_stationLoc[stationID][1])[:8],
    'city':'021',
}
params = parse.urlencode(params)
url = url + params
#print(url)
response = urllib.request.urlopen(url)
content = response.read().decode('utf-8')
data = json.loads(content)
data

In [14]:
stationID = id3 # "浦东国际机场（地铁站）
url='https://restapi.amap.com/v3/place/text?'
params = {
    'key': mykey,
    'keywords': "浦东国际机场（地铁站）",
    'types':150000,
    'location': str(dict_stationLoc[stationID][0])[:9] + ',' + str(dict_stationLoc[stationID][1])[:8],
    'city':'021',
}
params = parse.urlencode(params)
url = url + params
#print(url)
response = urllib.request.urlopen(url)
content = response.read().decode('utf-8')
data = json.loads(content)
data

{'suggestion': {'keywords': [], 'cities': []},
 'count': '31',
 'infocode': '10000',
 'pois': [{'parent': [],
   'address': '(在建)市域机场线后通段;2号线;市域机场线;磁悬浮',
   'distance': [],
   'pname': '上海市',
   'importance': [],
   'biz_ext': {'cost': [], 'rating': []},
   'biz_type': [],
   'cityname': '上海市',
   'type': '交通设施服务;地铁站;地铁站',
   'photos': [{'title': [],
     'url': 'https://aos-comment.amap.com/BV09301412/comment/464C0ADD_28FC_4B86_A06A_9B4DAF3A9D61_L0_001_2016_1512_1735888170153_29251497.jpg'},
    {'title': [],
     'url': 'http://store.is.autonavi.com/showpic/e0df4ac1be5f5d72b5825beddc1cce84'},
    {'title': [],
     'url': 'https://aos-comment.amap.com/BV09301412/comment/DA58FF4A_1226_49D3_A377_6682C620CF4D_L0_001_1500_2000_1735888170154_43381585.jpg'}],
   'typecode': '150500',
   'shopinfo': '2',
   'poiweight': [],
   'childtype': [],
   'adname': '浦东新区',
   'name': '浦东1号2号航站楼(地铁站)',
   'location': '121.805591,31.150958',
   'tel': [],
   'shopid': [],
   'id': 'BV09301412'},
  {'p

In [15]:
dict_stationPOI[id1] = 'BV10039895'
dict_stationPOI[id2] = 'BV10681608'
dict_stationPOI[id3] = 'BV09301412'

In [16]:
with open('TempData/dict_stationPOI_Recorrect.pkl', 'wb') as f:
    pickle.dump(dict_stationPOI, f)

***

In [17]:
def getTravelTime(origin_ID, destination_ID, originpoi, destinationpoi, dict_stationLoc):
    url = 'https://restapi.amap.com/v5/direction/transit/integrated?'
    params = {
        'key': LBSkey,
        'originpoi': originpoi,
        'destinationpoi': destinationpoi,
        'origin': str(dict_stationLoc[origin_ID][0])[:9] + ',' + str(dict_stationLoc[origin_ID][1])[:8],
        'destination': str(dict_stationLoc[destination_ID][0])[:9] + ',' + str(dict_stationLoc[destination_ID][1])[:8],
        'strategy': 6,
        'nightflag': 0,
        'date': '2017-08-31',
        'city1': '021',
        'city2': '021',
        'show_fields': 'cost'
    }
    
    params = parse.urlencode(params)
    url = url + params
    #print(url)
    response = urllib.request.urlopen(url)
    content = response.read().decode('utf-8')
    data = json.loads(content)


    return data

In [18]:
from tqdm import tqdm

with open('TempData/dict_stationPOI_Recorrect.pkl', 'rb') as f:
    dict_stationPOI = pickle.load(f)
    
problematic_stationPair = []
travelTime = np.zeros((len(stationID_List), len(stationID_List)))
travelDis = np.zeros((len(stationID_List), len(stationID_List)))

# Route Planning
for i in tqdm(range(len(stationID_List))):
    for j in range(i, len(stationID_List)):
        if i==j:
            continue
        origin_ID = stationID_List[i]
        destination_ID = stationID_List[j]
      
        origin_POI = dict_stationPOI[origin_ID]
        destination_POI = dict_stationPOI[destination_ID]
        TravelPlanFeedback = getTravelTime(origin_ID, destination_ID, origin_POI, destination_POI, dict_stationLoc)

        if TravelPlanFeedback['status'] == '1': # 抓取成功
            duration_pool = []   
            for travelPlan  in TravelPlanFeedback['route']['transits']:
                duration_pool.append(travelPlan['cost']['duration'])
            if len(duration_pool) != 0:  # 成功找到路径
                min_duration = min(duration_pool)
                idx = duration_pool.index(min_duration)
                travelTime[i][j] = min_duration
                travelTime[j][i] = min_duration
                distance = TravelPlanFeedback['route']['transits'][idx]['distance']
                travelDis[i][j] = distance
                travelDis[j][i] = distance

            else: # 未找到路径
                problematic_stationPair.append((origin_ID, destination_ID))
                print('Problematic Station Pair:', dict_staionName[origin_ID], dict_staionName[destination_ID])
                travelTime[i][j]=-1
                travelTime[j][i]=-1
                
        else: #抓取失败
            problematic_stationPair.append((origin_ID, destination_ID))
            print('Problematic Station Pair:', dict_staionName[origin_ID], dict_staionName[destination_ID])
            travelTime[i][j]=-1
            travelTime[j][i]=-1

print('Successfully get all travel time & distance')

with open('TempData/travelTime.pkl', 'wb') as f:
    pickle.dump(travelTime, f)

with open('TempData/travelDis.pkl', 'wb') as f:
    pickle.dump(travelDis, f)

with open('TempData/problematic_stationPair.pkl', 'wb') as f:   
    pickle.dump(problematic_stationPair, f)

  0%|          | 0/302 [00:00<?, ?it/s]

  4%|▎         | 11/302 [11:20<4:57:10, 61.27s/it]

Problematic Station Pair: 虹桥1号航站楼 远东大道
Problematic Station Pair: 虹桥1号航站楼 海天三路
Problematic Station Pair: 虹桥1号航站楼 浦东国际机场


  4%|▍         | 12/302 [12:20<4:54:23, 60.91s/it]

Problematic Station Pair: 上海动物园 野生动物园


...

Problematic Station Pair: 大世界 老西门


 73%|███████▎  | 220/302 [2:26:09<23:50, 17.44s/it]

Problematic Station Pair: 松江南站 松江大学城


100%|██████████| 302/302 [2:37:35<00:00, 31.31s/it]

Successfully get all travel time & distance


In [19]:
print(travelTime)

[[   0. 2384. 1406. ... 2961.  984. 1980.]
 [2384.    0. 1496. ... 4785. 2489. 1601.]
 [1406. 1496.    0. ... 3658. 1192. 1216.]
 ...
 [2961. 4785. 3658. ...    0. 3716. 4823.]
 [ 984. 2489. 1192. ... 3716.    0. 1601.]
 [1980. 1601. 1216. ... 4823. 1601.    0.]]


In [21]:
print(len(problematic_stationPair))
print(problematic_stationPair)
print( len(problematic_stationPair)/len(stationID_List)/len(stationID_List) )

58
[(1043, 261), (1043, 262), (1043, 263), (1044, 1628), (1045, 262), (1045, 263), (1020, 263), (1142, 263), (1323, 263), (1046, 262), (1046, 263), (1226, 262), (1321, 263), (1150, 263), (259, 927), (260, 927), (260, 2013), (261, 918), (261, 919), (261, 920), (261, 921), (261, 922), (261, 924), (261, 923), (261, 925), (261, 926), (261, 927), (261, 928), (261, 2013), (262, 931), (262, 918), (262, 919), (262, 920), (262, 921), (262, 922), (262, 924), (262, 929), (262, 930), (262, 923), (262, 925), (262, 926), (262, 928), (262, 2013), (262, 2014), (262, 2027), (263, 1326), (263, 314), (263, 931), (263, 932), (263, 413), (263, 930), (263, 2037), (263, 1019), (263, 2025), (263, 2029), (263, 1018), (834, 2008), (918, 922)]
0.0006359370203061269
